# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('data/clinical_trial_raw.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [2]:
# TODO: Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime
# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
# 3. Ensure all numeric columns are proper numeric types
# 4. Display the updated dtypes using df.dtypes
df = transform_types(df, type_map = {'enrollment_date' : 'datetime'})
# Update site and intervention_group columns before changing to categorical
df['site'] = df['site'].str.lower()
df['site'] = df['site'].str.strip()
df['site'] = df['site'].str.replace('  ', ' ')
df['site'] = df['site'].str.replace('_', ' ')

df['intervention_group'] = df['intervention_group'].str.lower()
df['intervention_group'] = df['intervention_group'].str.strip()
df['intervention_group'] = df['intervention_group'].str.replace('  ', ' ')
df['intervention_group'] = df['intervention_group'].str.replace('_', ' ')
df['intervention_group'] = df['intervention_group'].str.replace('treatmenta', 'treatment a')
df['intervention_group'] = df['intervention_group'].str.replace('treatmen a', 'treatment a')
df['intervention_group'] = df['intervention_group'].str.replace('contrl', 'control')


categorical_cols = ['site', 'intervention_group', 'sex']
for col in categorical_cols:
    df[col] = df[col].astype('category')
print(df.dtypes)

patient_id                    object
age                            int64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object


## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [3]:
# TODO: Calculate cholesterol ratio
df['cholesterol_ratio'] = df['cholesterol_ldl'] / df['cholesterol_hdl']

In [4]:
# TODO: Categorize blood pressure
df = create_bins(df,
    'systolic_bp',
    bins=[0, 120, 130, 999],
    labels=['Normal', 'Elevated', 'High'],
    new_column = 'bp_category'
)

**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [5]:
# TODO: Create age groups
df = create_bins(df,
    'age',
    bins=[0, 40, 55, 70, 100],
    labels=['<40', '40-54', '55-69', '70+'],
    new_column = 'age_group'
)

In [6]:
# TODO: Create BMI categories
df = create_bins(df,
    'bmi',
    bins=[0, 18.5, 24.9, 29.9, 999],
    labels=['Underweight', 'Normal', 'Overweight', 'Obese'],
    new_column = 'bmi_category'
)

## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [7]:
# TODO: String cleaning
for col in df:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip().str.lower()

## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [8]:
# TODO: One-hot encoding
df_int_group = pd.get_dummies(df['intervention_group'], dtype = 'int64')
df.drop('intervention_group', axis=1, inplace=True)
df_transformed = pd.concat([df, df_int_group], axis=1)
df_site = pd.get_dummies(df['site'], prefix='site', dtype = 'int64')
df.drop('site', axis=1, inplace=True)
df_transformed = pd.concat([df_transformed, df_site], axis=1)
print(df_transformed.shape, df_transformed.columns.tolist())

(10000, 29) ['patient_id', 'age', 'sex', 'bmi', 'enrollment_date', 'systolic_bp', 'diastolic_bp', 'cholesterol_total', 'cholesterol_hdl', 'cholesterol_ldl', 'glucose_fasting', 'site', 'follow_up_months', 'adverse_events', 'outcome_cvd', 'adherence_pct', 'dropout', 'cholesterol_ratio', 'bp_category', 'age_group', 'bmi_category', 'control', 'treatment a', 'treatment b', 'site_site a', 'site_site b', 'site_site c', 'site_site d', 'site_site e']


## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [9]:
# TODO: Save transformed data
df_transformed.to_csv('output/q6_transformed_data.csv', index=False)
